In [1]:
import os
import glob
import json
import copy
import re
import threading

In [2]:
domains = 'com|co.in|com.hk|de||co.uk|co.jp|fr|com.br|it|ru|es|ca|com.mx|co.id|com.tr|com.au|pl|com.sa|nl|com.ar|com.eg|co.th|com.pk|co.za|com.my|be|gr|com.vn|co.ve|com.tw|com.ua|at|se|com.co|ro|ch|pt|com.ph|cl|com.ng|com.sg|com.pe|ae|co.kr|co.hu|ie|dk|no|co.il|fi|cz|co.ma|sk|co.nz|com.kw|lk|bg|com.qa|az|kz|com.do|hr|by|com.ec|lt|iq|co.ke|com.bd|com.om|tn|si|co.cr|com.gt|com.pr|com.sv|lv|com.uy|jo|com.bo|ba|com.cu|rs|com.ly|cm|ee|co.ug|com.bh|com.np|com.gh|dz|lu|com.lb|co.uz|ci|com.py|com.ni|hn|com.et|tt|co.tz|mg|sn|cd|com.kh|am|com.af|ge|mu|com.bn|co.mz|com.jm|com.gi|is|com.pa|md|ps|com.na|mn|com.mt|co.bw|bj|kg|ml|rw|co.zm|bs|ht|la|com.bz|co.zw|as|cat|mk|ne|mw|tg|co.ao|gp|gy|bf|ga|li|co|tm|dj|mv|hk|sc|dm|bi|co.vi|vu|ad|com.vc|com.ag|com.fj|to|cf|fm|tk|gg|ws|vg|im|nu|gm|je|ms|me|co.im|tl|com.ai|gl|co.ls|co.je|st|it.ao|com.by|com.tj|pn|sh|com.sl|nr|sm|cg|co.ck|com.sb|com.cy|so|com.nf|com.ve|com.iq|jp|ac|com.tn|in|td'

pattern = u'^http[s]*://(www.|www.clck.|ru.|go.|nova.|m.|r.search.|)(?P<ref>[\w]+).(%s)(?P<tail>[\w/=%%-]*)'%domains

var_values = 'country|illness|direction|procedure|operation|city|%page'
one_var_pattern = '(?P<head>/(clinics|treatment|doctors|cities|page)/)(?P<var>(%s))=[\w]*'%var_values
two_var_pattern = '(?P<head>/(clinics|treatment)/)(?P<var1>(%s))=[\w]*/(?P<var2>(%s))=[\w]*'%(var_values, var_values)
object_pattern = '/(?P<var>(clinic|article|doctor|agency|program|diagnosticsKind|diagnostic|doc))/[\w-]*'
review_pattern = '/(?P<var>(review|diagnosticsKind|diagnostic|LK/request|clientLK/request|city|tprogram))/[0-9]*'

this_path = './sessions'
that_path = './paths'

In [3]:
def get_relation_mat(data_t):
    N=len(data_t)
    r  = re.compile('^http[s]*://[ru.]*bookimed.com(?P<path>/[\w/=-]*)')
    mat = [[0]*N]*N
    for i in range(0,N):
        for j in range(i+1,N):
            if data_t[j]['path']==data_t[i]['path'] and data_t[j]['referrer']==data_t[i]['referrer']:
                if data_t[j]['type']==3:
                    mat[i][j] = 2
                    continue
                if data_t[j]['type']==data_t[i]['type'] and data_t[i]['type']==1:
                    d1 = copy.deepcopy(data_t[i])
                    d2 = copy.deepcopy(data_t[j])
                    #d1 = data_t[i].values()
                    #d2 = data_t[j].values()
                    if d1.pop('timestamp')!=d2.pop('timestamp'):
                        d1.pop('userId')
                        d2.pop('userId')
                        try:
                            d1.pop('originalTimestamp')
                            d2.pop('originalTimestamp')
                        except:
                            pass
                        if d1==d2:
                            mat[i][j]=3
                            continue
                if data_t[j]['type']==2:
                    if data_t[j]['event']==2:
                        mat[i][j]=4
                        continue
                    if data_t[j]['event']==3:
                        mat[i][j]=5
                        continue
                    if data_t[j]['event']==1:
                        mat[i][j]=6
                        continue
            else:
                try:
                    ref = re.match(r, data_t[j]['referrer']).groupdict()['path'] 
                except:
                    ref = data_t[j]['referrer']
                if ref==data_t[i]['path']:
                    mat[i][j] = 1
                    continue
    return mat

In [4]:
def get_paths(mat_t):
    N = len(mat_t)
    paths = []
    if N-1 == 0:
        path = []
        if data[0]['type']==1:
            path+=[data[0]['path']]
            try:
                path+=[re.match(r, data[0]['referrer']).groupdict()['path'] ]
            except:
                path+=[data[0]['referrer']]
        if data[0]['type']==3:
            path+=['identify']
        paths+=[path]
    for j in range(N-1,0,-1):
        i=N-1
        path = []
        while (i>0) and (j>=1):
            #print j
            if sum([k[j] for k in mat_t[0:i]])==0:
                path += [data[j]['path']]
                path += [data[j]['referrer']]
                if j>1:
                    break
            for i in range(j-1,-1,-1):
                #print "\t %d" % i, mat_t[i][j]
                if mat_t[i][j]==0:
                    continue
                if mat_t[i][j] == 1:
                    path += [data[j]['path']]
                    break
                if mat_t[i][j] == 2:
                    path += ['identify']
                    break
                if mat_t[i][j] == 3:
                    #path += [data[j]['path']]
                    break
                    pass
                if mat_t[i][j] == 4:
                    path += [u'Requested Treatment']
                    break
                if mat_t[i][j] == 5:
                    path += [u'Requested Callback']
                    break
                if mat_t[i][j] == 6:
                    #print data[j]
                    path += [data[j]['properties']['button_type']]
                    break                  
            if i==0:
                if data[0]['type']==1:
                    path+=[data[0]['path']]
                    try:
                        path+=[re.match(r, data[0]['referrer']).groupdict()['path'] ]
                    except:
                        path+=[data[0]['referrer']]
                if data[0]['type']==3:
                    path+=['identify']
            j=i
            #print "\t \t" ,i,j, b
            #print (i!=0 or j!=1), '=', (i!=0), (j!=1)
        paths += [path]
    return paths

In [5]:
def filter_paths(paths_t):
    inds=[]
    paths=sorted(paths_t, key=lambda x: len(x),reverse=True)
    for i in range(len(paths)-1,-1,-1):
        for j in range(i-1,-1,-1):
            l = len(paths[j]) - len(paths[i])
            if paths[j][l:]==paths[i]:
                inds+=[i]
                break
    for i in inds:
        tenp=paths.pop(i)
    return paths

In [6]:
def simplify_path(paths_t):
    for path in paths_t:
        for i in range(0, len(path)):
            try:
                ref = re.match(pattern, path[i]).groupdict()['ref']
                if ref == 'bookimed':
                    path[i] = re.match(pattern, path[i]).groupdict()['tail']
                else:
                    path[i] = ref
            except:
                pass
            try:
                temp = re.match(object_pattern, path[i]).groupdict()
                path[i] = '/' + temp['var'] +'/'+ temp['var'] + '_name/'
            except:
                pass
            try:
                temp = re.match(review_pattern, path[i]).groupdict()
                path[i] = '/' + temp['var'] +'/num/'
            except:
                pass        
            try:
                temp = re.match(two_var_pattern, path[i]).groupdict()
                path[i] = temp['head']+temp['var1']+'/'+temp['var2']+'/'
            except:
                try:
                    temp = re.match(one_var_pattern, path[i]).groupdict()
                    path[i] = temp['head']+temp['var']+'/'
                except:
                    pass
            if '?' in str(path[i]):
                path[i] = path[i][:path[i].index('?')]
    return paths_t

In [7]:
def paths_to_str(paths_t):
    str_paths = []
    for i in paths_t:
        str_paths += [' , '.join(list(reversed(i)))]
    return str_paths

In [8]:
def paths_extract(l,r):
    for file_name in files[l:r]:
        f = open('%s/%s'%(this_path,file_name))
        context = f.readlines()
        data = []
        for i in context[:]:
            try:
                data += [json.loads(i.replace(',\n','').replace('\n',''))]
            except:
                pass
        data=sorted(data, key=lambda x: x['timestamp'])
        for path in paths_to_str(simplify_path(filter_paths(get_paths(get_relation_mat(data))))):
            output.write(path.encode('utf-8') + '\n')
            all_paths += [path]
        f.close()
    print "Done %d - %d" %(l,r)

In [9]:
files = [os.path.basename(x) for x in glob.glob('%s/*.json'%this_path)]

In [10]:
#all_paths = []
for file_name in files[:]:
    i_f = open('%s/%s'%(this_path, file_name))
    o_f = open('%s/%s'%(that_path, file_name), 'w+')
    context = i_f.readlines()
    data = []
    for i in context[:]:
        try:
            data += [json.loads(i.replace(',\n','').replace('\n',''))]
        except:
            pass
    data=sorted(data, key=lambda x: x['timestamp'])
    for path in paths_to_str(simplify_path(filter_paths(get_paths(get_relation_mat(data))))):
        o_f.write(path.encode('utf-8') + '\n')
        #all_paths += [path]
    #output.write('%s \n------------\n'%file_name)
    o_f.close()
    i_f.close()
    if files.index(file_name)%10000==0:
        print "+1"

+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1
+1


In [ ]:
ts = []
l_files = len(files)/100
N = 100
for i in range(0,N):
    ts += [threadinging.Thread(target=paths_extract, args=(l_files*i/N,l_files*(i+1)/N))]

for t in ts:
    t.start()

In [ ]:
output.close()

In [ ]:
len(files)

In [ ]:
len(all_paths)

In [ ]:
len(set(all_paths))

In [ ]:
srt = sorted(list(set(all_paths)), key = lambda x: all_paths.count(x), reverse=True)

In [ ]:
f = open('%s/%s'%(this_path,files[-5]))
context = f.readlines()
data = []
for i in context[:]:
    try:
        data += [json.loads(i.replace(',\n','').replace('\n',''))]
    except:
        pass
data=sorted(data, key=lambda x: x['timestamp'])

In [ ]:
paths_to_str(simplify_path(filter_paths(get_paths(get_relation_mat(data)))))

In [ ]:
data

In [ ]:
get_relation_mat(data)

In [ ]:
get_paths(get_relation_mat(data))

In [ ]:
filter_paths(get_paths(get_relation_mat(data)))